# 🧪Predicting Key Polymer Properties from Chemical Structure Using Machine Learning

This work focuses on **developing machine learning models** to predict **fundamental polymer properties** directly from their chemical structures represented as `SMILES` strings. By leveraging a large-scale open dataset and advanced regression techniques, the goal is to accurately forecast five critical polymer metrics: `density`, glass transition temperature (`Tg`), thermal conductivity (`Tc`), radius of gyration (`Rg`), and fractional free volume (`FFV`).

In [ ]:
!pip install /kaggle/input/rdkit-install-whl/rdkit_wheel/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl 

# 📚Imports and Library Setup
This section imports all the necessary libraries for data handling, visualization, preprocessing, modeling, evaluation, and other utilities used throughout the notebook.

In [ ]:
# Data handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib import gridspec
import seaborn as sns

# Scikit-learn: preprocessing, models, metrics, utilities
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, RepeatedKFold, learning_curve
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb

# Warnings
import warnings
warnings.filterwarnings("ignore")

#RDKit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors, Lipinski, rdMolDescriptors

# optuna
import optuna

#Rich
from rich.console import Console
from rich.table import Table

# 📥 Importing the Training Data¶
I load the datasets `train.csv` and `test.csv` using `pandas.read_csv()`. The data is comma-separated.

In [ ]:
# Load the training dataset from a CSV file
df_train = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train.csv", delimiter=',')

# Load the test dataset from a CSV file
df_test = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/test.csv", delimiter=',')


In [ ]:
df_train.head(5)

# 🔍EDA

## 🧹Data Cleaning and Missing Values Analysis

In this step, we start by removing the `'id'` column from the training dataset since it typically does not carry predictive information.

Next, we display a summary of the training DataFrame, which includes data types, non-null counts, and memory usage, to get a general overview of the dataset.

We then define a helper function `show_null` to calculate and print the percentage of missing values in each column.

In [ ]:
# Drop the 'id' column from the training DataFrame
df_train = df_train.drop("id", axis=1)

# Display summary information about the DataFrame (column types, non-null counts, memory usage)
df_train.info()

# Define a function to display the percentage of missing values in each column
def show_null(df):
    null_stats = pd.DataFrame({
        '%NaN': df.isna().mean() * 100  # Calculate percentage of missing values per column
    })
    print(null_stats)

# Separator for better readability in output
print("-------------------------------------------------------")

# Show missing value statistics for the training DataFrame
show_null(df_train)


## 📝Information about features:
* **id** - Unique identifier for each polymer.
* **SMILES** - Sequence-like chemical notation representing polymer structures.
* **Tg** - Glass transition temperature (°C). **Target variable**.
* **FFV** - Fractional free volume. **Target variable**.
* **Tc** - Thermal conductivity (W/m·K). **Target variable**.
* **Density** - Polymer density (g/cm³). **Target variable**.
* **Rg** - Radius of gyration (Å). **Target variable**.

In [ ]:
# Load supplemental dataset containing 'TC_mean' values (related to critical temperature)
train_supplement_Tc = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv", delimiter=',')

# Rename the 'TC_mean' column to 'Tc' for consistency with the main dataset
train_supplement_Tc['Tc'] = train_supplement_Tc['TC_mean']

# Drop the original 'TC_mean' column after renaming
train_supplement_Tc = train_supplement_Tc.drop("TC_mean", axis=1)

# Load supplemental dataset containing SMILES representations
train_supplement_SMILES = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv", delimiter=',')

# Load supplemental dataset containing 'Tg' values (e.g., glass transition temperatures)
train_supplement_Tg = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv", delimiter=',')

# Load supplemental dataset containing 'FFV' values (e.g., fractional free volume)
train_supplement_FFV = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv", delimiter=',')

This function groups a DataFrame by a specified column and computes the mean of another specified column within each group.

- **Parameters:**
  - `df`: Input pandas DataFrame.
  - `group_by_column`: The column name to group the data by.
  - `value_column`: The column name for which to calculate the mean.

- **Returns:**
  - A new DataFrame containing the group identifiers and their corresponding mean values.

This is useful for aggregating data and summarizing values by categories or groups.

In [ ]:
def calculate_group_mean(df, group_by_column, value_column):
    """
    Groups the DataFrame by a specified column and calculates the mean of another column.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        group_by_column (str): Column to group by.
        value_column (str): Column for which the mean will be calculated.
    
    Returns:
        pd.DataFrame: A DataFrame with the group-by column and the corresponding mean values.
    """
    # Group and calculate the mean
    grouped_mean = df.groupby(group_by_column)[value_column].mean().reset_index()
    
    # Rename columns for clarity
    grouped_mean.columns = [group_by_column, value_column]
    
    return grouped_mean


This function takes a list of `(name, dataframe)` tuples and prints a well-formatted table showing the number of rows in each DataFrame.

- Utilizes the `rich` library's `Console` and `Table` for clear, colored output.
- Displays DataFrame names alongside their row counts with thousands separators.
- Helpful for quick inspection and comparison of dataset sizes during analysis.

In [ ]:
def print_row_counts(dfs):
    """
    Nicely prints the number of rows for each DataFrame with column headers.
    
    :param dfs: A list of tuples in the format (dataframe_name, dataframe)
    """
    console = Console()
    table = Table(
        title="Row Count per DataFrame",
        show_header=True,            # Show column headers
        header_style="bold magenta", # Style for headers
        highlight=True,
        show_lines=True              # Show lines between rows
    )
    
    # Add columns with headers
    table.add_column("DataFrame Name", style="cyan", justify="left")
    table.add_column("Row Count", style="green", justify="right")
    
    for name, df in dfs:
        table.add_row(name, f"{df.shape[0]:,}")
    
    console.print(table)

In [ ]:
# Create separate DataFrames for each target property, keeping only rows without missing values

# Extract 'SMILES' and 'Tg' columns, drop rows with NaN, and reset index
df_Tg = df_train[['SMILES', 'Tg']].dropna().reset_index(drop=True)

# Extract 'SMILES' and 'FFV' columns, drop rows with NaN, and reset index
df_FFV = df_train[['SMILES', 'FFV']].dropna().reset_index(drop=True)

# Extract 'SMILES' and 'Tc' columns, drop rows with NaN, and reset index
df_Tc = df_train[['SMILES', 'Tc']].dropna().reset_index(drop=True)

# Extract 'SMILES' and 'Density' columns, drop rows with NaN, and reset index
df_density = df_train[['SMILES', 'Density']].dropna().reset_index(drop=True)

# Extract 'SMILES' and 'Rg' columns, drop rows with NaN, and reset index
df_Rg = df_train[['SMILES', 'Rg']].dropna().reset_index(drop=True)

# Combine all DataFrames with their corresponding task name
dfs = [
    ('Tg_train', df_Tg),
    ('FFV_train', df_FFV),
    ('Tc_train', df_Tc),
    ('density_train', df_density),
    ('Rg_train', df_Rg)
]

In [ ]:
print_row_counts(dfs)

In [ ]:
# Concatenate the original training DataFrames with the supplemental datasets to augment data
# Add supplemental Tg data to the original Tg DataFrame, resetting the index
df_Tg = pd.concat([df_Tg, train_supplement_Tg], ignore_index=True)

# Add supplemental FFV data to the original FFV DataFrame, resetting the index
df_FFV = pd.concat([df_FFV, train_supplement_FFV], ignore_index=True)

# Add supplemental Tc data to the original Tc DataFrame, resetting the index
df_Tc = pd.concat([df_Tc, train_supplement_Tc], ignore_index=True)

In [ ]:
dfs = [('Tg_train', df_Tg),
      ('FFV_train', df_FFV),
      ('Tc_train', df_Tc),
      ('density_train', df_density),
      ('Rg_train', df_Rg)]

In [ ]:
print_row_counts(dfs)

In [ ]:
df_Tg = calculate_group_mean(df_Tg, 'SMILES', 'Tg')
df_FFV = calculate_group_mean(df_FFV, 'SMILES', 'FFV')
df_Tc = calculate_group_mean(df_Tc, 'SMILES', 'Tc')
df_density = calculate_group_mean(df_density, 'SMILES', 'Density')
df_Rg = calculate_group_mean(df_Rg, 'SMILES', 'Rg')

In [ ]:
dfs = [('Tg_train', df_Tg),
      ('FFV_train', df_FFV),
      ('Tc_train', df_Tc),
      ('density_train', df_density),
      ('Rg_train', df_Rg)]
print_row_counts(dfs)


## 📘 **Molecular Descriptor Feature Engineering**

This function generates a set of **physicochemical descriptors** from SMILES strings using RDKit. These descriptors can help machine learning models predict polymer properties such as **glass transition temperature (Tg)**, **density**, **free volume fraction (FFV)**, and more.

Below are the descriptors used:

* **MW (Molecular Weight)**: Heavier molecules often correlate with higher Tg and density.
* **LogP (Partition Coefficient)**: Reflects hydrophobicity; can affect FFV and solubility.
* **Rotatable Bonds**: More flexibility usually lowers Tg and increases conformational entropy.
* **TPSA (Topological Polar Surface Area)**: Indicates polarity and hydrogen bonding potential; affects intermolecular interactions and Tc.
* **FractionCSP3**: Fraction of sp³-hybridized carbon atoms; higher values imply more saturation and potentially less rigidity.
* **RingCount**: Number of rings; ring structures can increase rigidity and Tg.

These features are simple to compute yet capture **key aspects of molecular structure**, making them valuable for data-driven property prediction.

In [ ]:
def create_features(row):
    """Extract molecular descriptors from a single row containing a SMILES string."""
    mol = Chem.MolFromSmiles(row['SMILES'])  # Convert SMILES to RDKit molecule
    if mol is None:
        return pd.Series({
        'MW': 0, # Molecular Weight
        'LogP': 0, # Hydrophobicity (Octanol-Water Partition Coefficient)
        'RotBonds': 0, # Number of rotatable bonds
        'TPSA': 0, # Topological Polar Surface Area
        'FractionCSP3': 0, # Fraction of sp3 carbon atoms
        'RingCount': 0 # Number of ring structures
    })
    # Compute basic molecular descriptors
    return pd.Series({
        'MW': Descriptors.MolWt(mol), # Molecular Weight
        'LogP': Descriptors.MolLogP(mol), # Hydrophobicity (Octanol-Water Partition Coefficient)
        'RotBonds': Lipinski.NumRotatableBonds(mol), # Number of rotatable bonds
        'TPSA': Descriptors.TPSA(mol), # Topological Polar Surface Area
        'FractionCSP3': rdMolDescriptors.CalcFractionCSP3(mol), # Fraction of sp3 carbon atoms
        'RingCount': Lipinski.RingCount(mol) # Number of ring structures
    })

In [ ]:
feature_df = df_Tg.apply(create_features, axis=1)
df_Tg = pd.concat([df_Tg, feature_df], axis=1)

feature_df = df_Tc.apply(create_features, axis=1)
df_Tc = pd.concat([df_Tc, feature_df], axis=1)

feature_df = df_FFV.apply(create_features, axis=1)
df_FFV = pd.concat([df_FFV, feature_df], axis=1)

feature_df = df_density.apply(create_features, axis=1)
df_density = pd.concat([df_density, feature_df], axis=1)

feature_df = df_Rg.apply(create_features, axis=1)
df_Rg = pd.concat([df_Rg, feature_df], axis=1)

In [ ]:
df_Tg.head(5)

## 🔬Converting SMILES Strings to Morgan Fingerprints

This code defines a function to convert molecular SMILES strings into fixed-length Morgan fingerprints using RDKit:

- Initializes a Morgan fingerprint generator with radius 2 and 1024 bits.
- `smiles_to_fp` function:
  - Takes a SMILES string and an RDKit fingerprint generator.
  - Converts the SMILES to an RDKit molecule object.
  - Returns a zero vector if the SMILES is invalid.
  - Otherwise, generates the fingerprint and returns it as a pandas Series.

Morgan fingerprints are widely used to represent molecular structures numerically for machine learning.


In [ ]:
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Parameters for fingerprint generation
NBITS = 1024

# Initialize the Morgan fingerprint generators
generator_r2 = GetMorganGenerator(radius=2, fpSize=NBITS)
#generator_r3 = GetMorganGenerator(radius=3, fpSize=NBITS)

def smiles_to_fp(smiles, generator):
    """
    Converts a SMILES string to a Morgan fingerprint using a pre-initialized RDKit generator.
    
    Parameters:
        smiles (str): SMILES representation of the molecule.
        generator: An RDKit fingerprint generator (e.g., from GetMorganGenerator).
        
    Returns:
        pd.Series: Fingerprint as a pandas Series. Returns zeros if SMILES is invalid.
    """
    # Convert SMILES to RDKit molecule
    mol = Chem.MolFromSmiles(smiles)
    
    # Return a zero vector if the SMILES is invalid
    if mol is None:
        return np.zeros(NBITS)
    
    # Generate the fingerprint and convert it to a NumPy array
    fp = generator.GetFingerprint(mol)
    return pd.Series(np.array(fp))

In [ ]:
def create_df(df):
    feature_df_r2 = df['SMILES'].apply(lambda x: smiles_to_fp(x, generator_r2))
    #feature_df_r3 = df['SMILES'].apply(lambda x: smiles_to_fp(x, generator_r3))
    feature_df_r2.columns = [f'bit_{i+1}' for i in range(NBITS)]
    #feature_df_r3.columns = [f'bit_{i+NBITS+1}' for i in range(NBITS)]
    
    df = df.drop('SMILES', axis = 1)
    # Concatenate features with original DataFrame
    return pd.concat([feature_df_r2, df.reset_index(drop=True)], axis=1)

df_Tg = create_df(df_Tg)

df_FFV = create_df(df_FFV)

df_Tc = create_df(df_Tc)

df_density = create_df(df_density)

df_Rg = create_df(df_Rg)

In [ ]:
df_FFV.head(5)

In [ ]:
X_Tg = df_Tg.drop("Tg", axis = 1)
X_FFV = df_FFV.drop("FFV", axis = 1)
X_Tc = df_Tc.drop("Tc", axis = 1)
X_Density = df_density.drop("Density", axis = 1)
X_Rg = df_Rg.drop("Rg", axis = 1)

In [ ]:
Y_Tg = df_Tg['Tg']
Y_FFV = df_FFV['FFV']
Y_Tc = df_Tc['Tc']
Y_Density = df_density['Density']
Y_Rg = df_Rg['Rg']

## 🔀 Splitting the Dataset

I separate the features (`Xdata`) and labels (`Ydata`) for model training.

- The `Personality` column is selected as the target.
- The rest of the DataFrame is used as input features.

I then split the dataset into:

- **Training set (50%)**
- **Validation set (25%)**
- **Test set (25%)**

In [ ]:
def spit_data(name, Xdata, Ydata, random_seed = 42):
    # Split into training (80%) and test (20%)
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xdata, Ydata, test_size=0.2, random_state=random_seed)

    print(f"----------------split for {name}_df----------------")
    # Print shapes of the splits
    print(f"Train shape, X: {Xtrain.shape}, y: {Ytrain.shape}")
    print(f"Test shape, X: {Xtest.shape}, y: {Ytest.shape}")
    print()
    return Xtrain, Xtest, Ytrain, Ytest
    

In [ ]:

X_Tg_Train, X_Tg_Test, Y_Tg_Train, Y_Tg_Test = spit_data("Tg" ,X_Tg, Y_Tg)

X_FFV_Train, X_FFV_Test, Y_FFV_Train, Y_FFV_Test = spit_data("FFV" ,X_FFV, Y_FFV)

X_Tc_Train, X_Tc_Test, Y_Tc_Train, Y_Tc_Test = spit_data("Tc" ,X_Tc, Y_Tc)

X_Density_Train, X_Density_Test, Y_Density_Train, Y_Density_Test = spit_data("density" ,X_Density, Y_Density)

X_Rg_Train, X_Rg_Test, Y_Rg_Train, Y_Rg_Test = spit_data("Rg" ,X_Rg, Y_Rg)

## 📊Visualizing Target Variable Distributions with Histograms and Mean Lines

This code creates a multi-plot figure to visualize the distributions of different target variables in the training set:

- Uses `seaborn` to plot histograms with KDE overlays for each target.
- Adds a vertical dashed line marking the mean value of each distribution.
- Places the mean value as a text label below the line for clarity.
- Organizes subplots in a grid layout, dedicating the bottom row to the 'FFV' target spanning both columns.
- Helps in understanding the spread and central tendency of each target variable's training data.


In [ ]:
fig = plt.figure(figsize = (12,15), constrained_layout = True)
spec = gridspec.GridSpec(nrows = 3, ncols = 2, figure = fig)

dfs = [('Tg', Y_Tg_Train),
      ('Rg', Y_Rg_Train),
      ('Tc', Y_Tc_Train),
      ('Density', Y_Density_Train),
      ('FFV', Y_FFV_Train)]

i = 0
for name, df in dfs:
    if name == 'FFV':
        ax = fig.add_subplot(spec[2, :])
    else:
        ax = fig.add_subplot(spec[i // 2, i % 2])

    mean = df.mean()

    sns.histplot(df, kde = True, label = 'Histogram', bins = 14, color = "cornflowerblue")
    ax.axvline(mean, color = 'darkorchid', linestyle = "--", linewidth = 2, label = 'Mean')

    ax.text(mean, plt.ylim()[1]*(-0.1), f'{mean:.2f}', 
         ha='center', va='bottom', color='darkorchid',
         bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

    ax.set_title(f"Distribution of {name}(Train)")
    ax.set_xlabel('Value')
    ax.set_ylabel('Count/Density')

    ax.grid(True)
    ax.legend()
    i+=1

plt.show()

## 📈 Target Variable Distributions – Observations

The following plots show the distributions of the five target properties from the training set:

1. **Tg (Glass Transition Temperature)**
- Appears slightly **right-skewed**, with a wide spread from negative to ~500.
- The mean value is **~100.6**, marked by the purple dashed line.
- Indicates significant variability in thermal behavior among the polymers.

2. **Rg (Radius of Gyration)**
- Also **right-skewed**, with values primarily concentrated between 10 and 25.
- The mean is around **16.4**.
- The tail suggests some polymers have relatively high chain flexibility or extension.

3. **Tc (Crystallization Temperature)**
- Strong **positive skew** with most values clustered between 0.1 and 0.4.
- Mean value is approximately **0.26**.
- Many polymers do not crystallize easily or crystallize at low temperatures.

4. **Density**
- Shows a **right-skewed** distribution centered near **0.99**.
- Most values are between 0.8 and 1.1, typical for organic polymer materials.
- The long tail toward higher densities might reflect highly packed or crosslinked structures.

5. **FFV (Fractional Free Volume)**
- **Approximately normal distribution** centered at **0.37**.
- Symmetric shape suggests more stable variance in FFV across polymers.
- FFV may be a more predictable target for regression tasks compared to others.

## 📊Normality Assessment of FFV Target Variable

The **QQ-plot** visualizes how closely the fractional free volume (`FFV`) training data follows a normal distribution. Points aligning along the reference line indicate normality. 

Additionally, the Shapiro-Wilk test statistically evaluates the normality of the data. A high `p-value` (typically > 0.05) suggests that the null hypothesis of normality cannot be rejected, implying the data is approximately normally distributed. Conversely, a low p-value indicates deviation from normality.

In [ ]:
import scipy.stats as stats
import statsmodels.api as sm

sm.qqplot(Y_FFV_Train, line='s')
plt.title("QQ-Plot for FFV")
plt.show()

# Shapiro-Wilk Test
shapiro_stat, shapiro_p = stats.shapiro(Y_FFV_Train)
print(f"Shapiro-Wilk test: statistic={shapiro_stat:.4f}, p-value={shapiro_p}")


🧪 **Shapiro–Wilk Test Interpretation:**

* **Test Statistic:** 0.9028
* **p-value:** 3.41 × 10⁻⁵⁷

The Shapiro–Wilk test checks the null hypothesis that the data is drawn from a **normal distribution**.

Since the **p-value is extremely small (much less than 0.05)**, we **strongly reject H₀**.
This means that **the distribution of FFV is not normal**.

🔍 **What the Q-Q plot shows**:
The center of the distribution (near 0 on the x-axis) lies almost perfectly on the red line - indicating a good fit to the normal distribution in the middle part.

The tails deviate a bit:

* Left: the lower quantiles are slightly lower than the theoretical ones → a small "left tail".

* Right: upper quantiles are strongly above the theoretical ones → right "heavy" tail (typical case of slight asymmetry or right "fat" side).

## 📊Distribution Analysis of Numeric Features in Training Data

This visualization presents `histograms` with **KDE** overlays for several numeric feature columns combined from different training subsets:

- Shows the distribution of molecular descriptors like `molecular weight` (MW), `LogP`, `number of rotatable bonds`, `TPSA`, `fraction of sp3 carbons`, and `ring count`.
- Marks the mean value for each feature with a dashed red vertical line and annotates the exact mean below the line.
- Arranged in a grid layout to easily compare distributions side-by-side.
- Useful for understanding feature ranges and central tendencies across the combined training data.


In [ ]:
Xtrain_combined = pd.concat([X_Tg_Train, X_FFV_Train, X_Tc_Train, X_Density_Train, X_Rg_Train])

fig = plt.figure(figsize = (15,13), constrained_layout = True)

spec = gridspec.GridSpec(nrows = 3, ncols = 2, figure = fig)

target_cols = ['MW', 'LogP', 'RotBonds', 'TPSA', 'FractionCSP3', 'RingCount']

for i, col in enumerate(target_cols):
    ax = fig.add_subplot(spec[i//2, i%2])

    mean = Xtrain_combined[col].mean()
    sns.histplot(Xtrain_combined[col], kde=True, label = 'Histogram', bins = 14, color = "darkorange")

    ax.axvline(mean, linestyle ='--', color = "red", linewidth = 2, label = 'Mean')

    ax.text(mean, plt.ylim()[1]*(-0.15), f'{mean:.2f}', 
         ha='center', va='bottom', color='red',
         bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

    ax.set_title(f"Distribution of {col}(Train)")

    ax.set_xlabel('Value')
    ax.set_ylabel('Count/Density')

    ax.grid(True)
    ax.legend()

plt.show()

## 📈 Molecular Feature Distributions — Summary & Insights

The histograms above represent the distributions of several molecular descriptors extracted from SMILES strings using RDKit. These descriptors can provide valuable structural and physicochemical information for polymer property prediction tasks (e.g. Tg, Tc, Density, FFV, Rg). Here's what we observe:

1. **Molecular Weight (MW)**
- Distribution is **right-skewed**, with most molecules below ~600 g/mol.
- A long tail suggests presence of heavier, possibly more complex structures.
- Mean ≈ **423.6**.

2. **LogP (Octanol-Water Partition Coefficient)**
- Appears to follow a **nearly normal distribution**, centered around ~5.5.
- Suggests a balanced lipophilic/hydrophilic profile in many compounds.
- Extreme LogP values may indicate unusual solubility characteristics.

3. **Rotatable Bonds (RotBonds)**
- Strong **right skew** — most molecules have < 10 rotatable bonds.
- A few outliers (up to ~80) indicate flexible or large chain-like structures.
- Flexibility can correlate with properties like Tg and Rg.
  
4. **Topological Polar Surface Area (TPSA)**
- Again, **right-skewed** with many small values and a few high ones (>200).
- High TPSA may indicate increased polarity or hydrogen bonding potential.
- Mean ≈ **62**, which is relatively moderate.

5. **Fraction of sp³ Carbons (FractionCSP3)**
- Shows a **bimodal trend**, with peaks near 0 and 1.
- Indicates presence of both highly aromatic and fully saturated compounds.
- May relate to polymer rigidity and branching.

6. **Ring Count**
- Majority of molecules have **fewer than 5 rings**, with a sharp drop-off.
- Right-skewed again, with occasional polycyclic compounds.
- Rings influence molecular rigidity, possibly impacting properties like density or FFV.

## 🔗Correlation Matrix of Numeric Features

In this step, we calculate the **correlation matrix** for the selected numeric target columns in the training dataset. Correlation measures the strength and direction of the linear relationship between pairs of variables.

- `Xtrain_combined[target_cols].corr()` computes the correlation matrix.
- We then visualize the matrix using a **heatmap** with `seaborn`:
  - `annot=True` displays the correlation values on the heatmap.
  - `cmap="coolwarm"` applies a diverging color palette for better interpretation.
  - `fmt=".2f"` formats the correlation values to two decimal places.

This visualization helps identify strong positive or negative correlations between features, which can be useful for feature selection or multicollinearity analysis.


In [ ]:
corr = Xtrain_combined[target_cols].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrix of correlations of numeric features")
plt.show()

## 🔗Correlation of Numeric Features with Target Variables

In this section, we compute and display the correlation between several **numeric features** and different **target variables**. The results are presented in a formatted table using the **Rich** library for better visualization in the console.

Steps:
- Define:
  - `targets`: list of target variables (e.g., Tg, Tc, FFV, etc.).
  - `numeric_cols`: list of numeric feature columns.
  - `dfs_X`: corresponding feature datasets for each target.
  - `Ys`: corresponding target series for each dataset.
- Create a Rich `Table` with:
  - A title and headers for the target and feature names.
  - Color styling (`cyan` for targets, `green` for features).
- For each target:
  - Compute Pearson correlation (`df_X[col].corr(Y)`) between each numeric feature and the target.
  - Add the results as a row in the table.
- Print the table using `console.print()`.

This table provides a quick comparison of how strongly each numeric feature correlates with each target, which is helpful for **feature importance assessment** and **model input selection**.


In [ ]:
targets = ['Tg', 'Tc', 'FFV', 'Density', 'Rg']
numeric_cols = ['MW', 'LogP', 'RotBonds', 'TPSA', 'FractionCSP3', 'RingCount']
dfs_X = [X_Tg_Train, X_Rg_Train, X_Tc_Train, X_Density_Train, X_FFV_Train]
Ys = [Y_Tg_Train, Y_Rg_Train, Y_Tc_Train, Y_Density_Train, Y_FFV_Train]

console = Console()
table = Table(
    title="Correlations of numeric features with target variables",
    show_header=True,
    header_style="bold magenta",
    highlight=True,
    show_lines=True
)
table.add_column("Target Name", style="cyan", justify="left")

for col_name in numeric_cols:
    table.add_column(col_name, style="green", justify="right")

# Cycle through targets and datasets
for target_name, df_X, Y in zip(targets, dfs_X, Ys):
    correlations = [f"{df_X[col].corr(Y):.3f}" for col in numeric_cols]
    table.add_row(target_name, *correlations)

console.print(table)


# ⚙️Models tuning

## 🎯Objective Function for XGBoost Hyperparameter Optimization (Optuna)

This function defines the **objective function** for hyperparameter tuning of an XGBoost regressor using **Optuna**. The goal is to optimize model parameters to minimize the cross-validated error.

Key steps:
- **Parameter Sampling**: Using `trial.suggest_*` methods, we define the search space for:
  - `n_estimators`: number of boosting rounds.
  - `max_depth`: depth of each tree.
  - `learning_rate`: shrinkage factor for weights (sampled logarithmically).
  - `subsample`: fraction of samples used for each tree.
  - `colsample_bytree`: fraction of features per tree.
  - `gamma`: minimum loss reduction for further partitioning.
  - `reg_alpha` and `reg_lambda`: L1 and L2 regularization terms.
  - `min_child_weight`: minimum sum of instance weights per child.
- **Model Setup**: Create an `XGBRegressor` with the sampled parameters.
- **Cross-Validation**:
  - Use `KFold` with 5 splits (shuffled) for performance evaluation.
  - Metric: `neg_mean_absolute_error` (negative MAE, as required by Optuna).
- **Scoring**: Uses negative mean absolute error (`neg_mean_absolute_error`) for Optuna compatibility.
- **Return Objective**: Mean cross-validation score is returned for optimization.

In [ ]:
def objective_xgb(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 600),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 5, 20),
        'reg_lambda': trial.suggest_float('reg_lambda', 5, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 15),
        'random_state': 42,
        'tree_method': 'hist'  
    }
    
    model = xgb.XGBRegressor(**params)

    #if X.shape[0] < 1000:
    #    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
    #else:
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=cv, scoring = 'neg_mean_absolute_error')

    return scores.mean()

## 🎯Objective Function for LightGBM Hyperparameter Optimization (Optuna)

This function defines the **objective function** for tuning a LightGBM regressor using **Optuna**. The goal is to find the best hyperparameters to minimize the cross-validated mean absolute error.

Highlights:
- **Hyperparameter Search Space** includes:
  - `n_estimators`: number of boosting iterations.
  - `max_depth`: maximum tree depth.
  - `learning_rate`: step size shrinkage (log scale).
  - `num_leaves`: maximum number of leaves in one tree.
  - `subsample` (bagging fraction) and `colsample_bytree` (feature fraction) for sampling data and features.
  - Regularization parameters `reg_alpha` and `reg_lambda` with log-scaled search.
  - `min_child_samples`: minimum number of data points in a leaf.
- **Model Setup**: Creates a `LGBMRegressor` with the sampled parameters, setting random seed and enabling parallel jobs.
- **Cross-Validation**: Uses 5-fold KFold with shuffling to evaluate performance.
- **Scoring**: Uses negative mean absolute error (`neg_mean_absolute_error`) for Optuna compatibility.
- **Return Objective**: Mean cross-validation score is returned for optimization.

In [ ]:
import lightgbm as lgb
def objective_lgbm(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 550),
        'max_depth': trial.suggest_int('max_depth', 3, 8),  
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 40, 100),  
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),  # bagging_fraction
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),  # feature_fraction
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-6, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-6, 10.0, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state': 42,
        'n_jobs': -1,
        'boosting_type': 'gbdt'
    }

    model = lgb.LGBMRegressor(**params, verbose = -1)

    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    
    return scores.mean()

## 🎯Support Vector Regression (SVR) with Validation Curves for Hyperparameter Tuning

This function evaluates the effect of the **regularization parameter `C`** on the performance of an SVR model with an RBF kernel, using different values of the **epsilon (`ε`)** parameter.

Key points:
- Data scaling:
  - The model is tested on both **unscaled** and **standard scaled** versions of the training data.
- `validation_curve` from `sklearn` is used to compute training and validation scores over a range of `C` values (`param_range`).
- `epsilon` controls the margin of tolerance where no penalty is given to errors.
- For each `epsilon` in `[0.0005, 0.001, 0.01, 0.1]`:
  - A subplot shows the validation curve plotting **MAE** (mean absolute error) against `C` on a logarithmic scale.
  - Both training and validation errors are displayed to visualize potential overfitting or underfitting.
- The plots help identify the optimal `C` value and the impact of data scaling on model performance.

This detailed visualization guides the selection of hyperparameters for SVR, improving generalization on unseen data.


In [ ]:
from sklearn.model_selection import validation_curve
def SVR_fit(Xtrain, Ytrain):
    scaler = StandardScaler()
    Xtrain_scaled = scaler.fit_transform(Xtrain)
    eps = [0.0005, 0.001, 0.01, 0.1]
    param_range = np.logspace(-4, 3, 10)
    Xdata = [('no scaler', Xtrain),
            ('standard scaler', Xtrain_scaled)]
    for name, X in Xdata:
        print(f"-------------------------------------with {name}-------------------------------------")
        fig = plt.figure(figsize=(15,13), constrained_layout = True)

        spec = gridspec.GridSpec(nrows = 2, ncols = 2, figure = fig)
        for i, epsilon in enumerate(eps):
            ax = fig.add_subplot(spec[i//2, i%2])
            model = SVR(kernel='rbf', epsilon = epsilon)
            train_scores, valid_scores = validation_curve(
                model, X, Ytrain,
                param_name="C",
                param_range=param_range,
                cv=5,  
                scoring="neg_mean_absolute_error"
            )
    
            train_scores_mean = -train_scores.mean(axis=1)
            valid_scores_mean = -valid_scores.mean(axis=1)
    
            ax.semilogx(param_range, train_scores_mean, label="Train", color="blue")
            ax.semilogx(param_range, valid_scores_mean, label="Validation", color="orange")
            ax.set_xlabel("C")
            ax.set_ylabel("MAE")
            ax.grid(True)
            ax.set_title(f"Validation curve for SVR(eps = {epsilon})")
            ax.legend()
        plt.show()


## 🎯Objective Function for Random Forest Hyperparameter Optimization (Optuna)

This function defines an **objective function** to optimize hyperparameters of a `RandomForestRegressor` using **Optuna**.

Details:
- **Hyperparameters tuned:**
  - `n_estimators`: number of trees in the forest.
  - `max_depth`: maximum depth of each tree.
  - `min_samples_split`: minimum samples required to split an internal node.
  - `min_samples_leaf`: minimum samples required at a leaf node.
  - `max_features`: number of features to consider when looking for the best split; chosen from categorical options (`sqrt`, `log2`, 0.8, 1).
- **Cross-validation strategy:**
  - If dataset is small (< 1000 samples), use `RepeatedKFold` with 5 splits and 3 repeats for more robust evaluation.
  - Otherwise, use standard shuffled `KFold` with 5 splits.
- **Scoring metric:** negative mean absolute error (`neg_mean_absolute_error`), compatible with Optuna.
- The function returns the mean cross-validation score, which Optuna tries to maximize.

This function supports systematic tuning of Random Forest parameters to improve regression performance.


In [ ]:
def objective_rf(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_samples_split': trial.suggest_int('min_samples_split', 5, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 5, 20),
        'max_features': trial.suggest_categorical('max_features', [ 'sqrt', 'log2', 0.8, 1]),
        'random_state': 42,
    }

    model = RandomForestRegressor(**params)

    if X.shape[0] < 1000:
        cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
    else:
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
            
    scores = cross_val_score(model, X, y, cv=cv, scoring = 'neg_mean_absolute_error')

    return scores.mean()

## 🧠Plotting Learning Curves to Monitor Overfitting and Model Performance

This function plots the **learning curve** of a regression model, showing how training and validation errors evolve as the size of the training data increases.

Key points:
- Uses 5-fold shuffled cross-validation to evaluate model performance at different training set sizes.
- Tracks **mean absolute error (MAE)** on both training and validation sets.
- The plot helps monitor **overfitting**:
  - If the training error is much lower than validation error and the gap doesn't close with more data, the model is likely overfitting.
  - If training and validation errors converge and remain low, the model generalizes well.
- Allows assessment of whether adding more training data could improve validation performance or if the model complexity needs adjustment.

This visualization is essential for ensuring the model is not overfitting and for guiding decisions on model tuning or data collection.

In [ ]:
def print_learning_curve(Xdata, Ydata, model, name):
    fig = plt.figure(figsize = (8,6), constrained_layout = True)
    
    cv = KFold(n_splits = 5, shuffle = True, random_state = 42)
    
    train_sizes, train_scores, val_scores = learning_curve(
        model,
        Xdata,
        Ydata,
        cv = cv,
        scoring = 'neg_mean_absolute_error',
        train_sizes = np.linspace(0.1,1.0,10),
        n_jobs = -1
    )

    train_scores_mean = -np.mean(train_scores, axis = 1)
    val_scores_mean = - np.mean(val_scores, axis = 1)

    plt.plot(train_sizes, train_scores_mean, 'o-', color = 'blue', label = 'Train error')
    plt.plot(train_sizes, val_scores_mean, 'o-', color = 'orange', label = 'Validation error')
    plt.xlabel('Training size')
    plt.ylabel('MAE')
    plt.title(f'Learning curve for {name}')
    plt.legend()
    plt.grid()

    plt.show()

## 🔥Tg + XGBoost

This code runs an Optuna study to maximize the cross-validated score by tuning XGBoost hyperparameters over 50 trials. After optimization, it prints the best-found parameters.

In [ ]:
study_xgb = optuna.create_study(direction = 'maximize')
study_xgb.optimize(lambda trial : objective_xgb(trial, X_Tg_Train, Y_Tg_Train), n_trials=50)
#'exact'
print("The best paramters for XGBoost:")
print(study_xgb.best_params)

An `XGBoost` regressor is instantiated using the best hyperparameters found earlier, trained on the training data, and the training mean absolute error (MAE) is printed to evaluate fit quality.

In [ ]:
model_Tg = xgb.XGBRegressor(
    n_estimators = 504,
    max_depth = 3,
    learning_rate = 0.022748903648379455,
    subsample =0.5150950623784505,
    colsample_bytree = 0.8136028949107683,
    gamma =  1.7296216677361642,
    reg_alpha =  7.759582933382447,
    reg_lambda = 10.899776530374172,
    min_child_weight = 4,
    random_state = 42
)

model_Tg.fit(X_Tg_Train, Y_Tg_Train)
print(mean_absolute_error(Y_Tg_Train, model_Tg.predict(X_Tg_Train)))

In [ ]:
print_learning_curve(X_Tg_Train, Y_Tg_Train, model_Tg, 'Tg')

## 🔥TC + SVR

This command runs the `SVR_fit` function on the training data for the target `Tc`. It generates validation curves for different values of the SVR `epsilon` parameter, comparing training and validation errors across a range of regularization strengths (`C`) with and without feature scaling.

In [ ]:
SVR_fit(X_Tc_Train, Y_Tc_Train)

The `SVR` model is trained using the parameters `C=50` and `epsilon=0.01`, which were chosen based on the validation curves.

In [ ]:
model_Tc = SVR(kernel='rbf', C = 115, epsilon = 0.01)
model_Tc.fit(X_Tc_Train, Y_Tc_Train)
print(mean_absolute_error(Y_Tc_Train, model_Tc.predict(X_Tc_Train)))

In [ ]:
print_learning_curve(X_Tc_Train, Y_Tc_Train, model_Tc, 'TC')

## 🔥FFV + LGBM

This code performs hyperparameter tuning for a `LightGBM` model over 50 trials to maximize validation performance. The best parameters found are then printed.

In [ ]:
study_lgbm = optuna.create_study(direction = 'maximize')
study_lgbm.optimize(lambda trial : objective_lgbm(trial, X_FFV_Train, Y_FFV_Train), n_trials=50)
#'exact'
print("The best paramters for LGBM:")
print(study_lgbm.best_params)

The `LightGBM` regressor is instantiated with the best-tuned parameters and trained on the **FFV** dataset. The training mean absolute error (MAE) is printed to assess model fit.

In [ ]:
model_FFV = lgb.LGBMRegressor(
    n_estimators = 548,
    max_depth = 8,
    learning_rate = 0.07285429805216179,
    num_leaves = 51,
    subsample =0.9186429709788774,
    colsample_bytree = 0.8451398289748779,
    reg_alpha =0.011786753567576962,
    reg_lambda =2.226162826179684e-08,
    min_child_samples = 11,
    random_state =  42,
    boosting_type = 'gbdt',
    verbose = -1
)

model_FFV.fit(X_FFV_Train, Y_FFV_Train)
print(mean_absolute_error(Y_FFV_Train, model_FFV.predict(X_FFV_Train)))

In [ ]:
print_learning_curve(X_FFV_Train, Y_FFV_Train, model_FFV, 'FFV')

## 🔥Density + LBGM

This code performs hyperparameter tuning for a `LightGBM` model over 50 trials to maximize validation performance. The best parameters found are then printed.

In [ ]:
study_lgb = optuna.create_study(direction = 'maximize')
study_lgb.optimize(lambda trial : objective_lgbm(trial, X_Density_Train, Y_Density_Train), n_trials=50)
#'exact'
print("The best paramters for LGBM:")
print(study_lgb.best_params)

The `LightGBM` regressor is instantiated with the best-tuned parameters and trained on the **Density** dataset. The training mean absolute error (MAE) is printed to assess model fit.

In [ ]:
model_Density = lgb.LGBMRegressor(
    n_estimators = 445,
    max_depth = 3,
    learning_rate = 0.03005975330148901,
    num_leaves = 78,
    subsample =0.7020942819679887,
    colsample_bytree = 0.7844462554278813,
    reg_alpha = 0.0004484916008774172,
    reg_lambda =0.0005698184260493566,
    min_child_samples = 5,
    random_state =  42,
    boosting_type = 'gbdt',
    verbose = -1
)

model_Density.fit(X_Density_Train, Y_Density_Train)
print(mean_absolute_error(Y_Density_Train, model_Density.predict(X_Density_Train)))

In [ ]:
print_learning_curve(X_Density_Train, Y_Density_Train, model_Density, 'Density')

## 🔥RG + LGBM

This code performs hyperparameter tuning for a `LightGBM` model over 50 trials to maximize validation performance. The best parameters found are then printed.

In [ ]:
study_lgb = optuna.create_study(direction = 'maximize')
study_lgb.optimize(lambda trial : objective_lgbm(trial, X_Rg_Train, Y_Rg_Train), n_trials=50)
#'exact'
print("The best paramters for LGBM:")
print(study_lgb.best_params)

The `LightGBM` regressor is instantiated with the best-tuned parameters and trained on the **Rg** dataset. The training mean absolute error (MAE) is printed to assess model fit.

In [ ]:
model_Rg = lgb.LGBMRegressor(
    n_estimators = 417,
    max_depth = 3,
    learning_rate = 0.0223696041817692,
    num_leaves = 85,
    subsample =0.6316470850414286,
    colsample_bytree = 0.8396247783719049,
    reg_alpha = 4.306437129417159e-06,
    reg_lambda =0.011339773998675985,
    min_child_samples = 7,
    random_state =  42,
    boosting_type = 'gbdt'
)

model_Rg.fit(X_Rg_Train, Y_Rg_Train)
print(mean_absolute_error(Y_Rg_Train, model_Rg.predict(X_Rg_Train)))

In [ ]:
print_learning_curve(X_Rg_Train, Y_Rg_Train, model_Rg, 'Rg')

In [ ]:
all_models = [model_Tg, model_FFV, model_Tc, model_Density, model_Rg]
col_names = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
all_X_Train = [X_Tg_Train, X_FFV_Train, X_Tc_Train, X_Density_Train, X_Rg_Train]
all_X_Test = [X_Tg_Test, X_FFV_Test, X_Tc_Test, X_Density_Test, X_Rg_Test]

all_Y_Train = [Y_Tg_Train, Y_FFV_Train, Y_Tc_Train, Y_Density_Train, Y_Rg_Train]
all_Y_Test = [Y_Tg_Test, Y_FFV_Test, Y_Tc_Test, Y_Density_Test, Y_Rg_Test]

## 📊Plotting Learning Curves for Multiple Models

This code generates learning curves for several models across different target variables, displaying how training and validation errors (MAE) evolve with increasing training set sizes.

Details:
- Uses 5-fold shuffled cross-validation for robust evaluation.
- Plots are arranged in a 3x2 grid for clear comparison.
- Each subplot shows training and validation MAE against training size for one model.
- Helps to visually assess model generalization and detect overfitting or underfitting across multiple targets at once.


In [ ]:
from sklearn.model_selection import learning_curve, KFold

fig = plt.figure(figsize = (18,15), constrained_layout = True)

spec = gridspec.GridSpec(nrows = 3, ncols = 2, figure = fig)
i = 0
for name, model, X_train, Y_train in zip(col_names, all_models, all_X_Train, all_Y_Train):
    cv = KFold(n_splits = 5, shuffle = True, random_state = 42)

    train_sizes, train_scores, val_scores = learning_curve(
        model,
        X_train,
        Y_train,
        cv = cv,
        scoring = 'neg_mean_absolute_error',
        train_sizes = np.linspace(0.1,1.0,10),
        n_jobs = -1
    )

    train_scores_mean = -np.mean(train_scores, axis = 1)
    val_scores_mean = - np.mean(val_scores, axis = 1)

    ax = fig.add_subplot(spec[i//2, i%2])

    ax.plot(train_sizes, train_scores_mean, 'o-', color = 'blue', label = 'Train error')
    ax.plot(train_sizes, val_scores_mean, 'o-', color = 'orange', label = 'Validation error')
    ax.set_xlabel('Training size')
    ax.set_ylabel('MAE')
    ax.set_title(f'Learning curve for {name}')
    ax.legend()
    ax.grid()
    i+=1

plt.show()

# ⚖️Computing Weighted Mean Absolute Error (wMAE) Across Multiple Models

This function calculates a **weighted mean absolute error (wMAE)** for a set of regression models and datasets, accounting for differences in sample sizes and target ranges.

Key points:
- For each target/property:
  - `n_i`: number of samples.
  - `r_i`: range of target values (max - min).
- Weights `w_i` are computed to balance the influence of each target, normalizing by range and adjusting for sample size.
- The total weighted absolute error is computed as the sum of weighted absolute errors from all models.
- Final wMAE normalizes the total weighted error by the total number of samples.
- Returns both the overall wMAE and the list of weights used.

In [ ]:
def compute_wmae(all_models, all_X_Data, all_Y_Data):
    K = len(all_models)
    
    # Counting n_i and r_i for each property
    n_values = [len(y) for y in all_Y_Data]
    r_values = [y.max() - y.min() for y in all_Y_Data]
    
    # Calculate the denominator for the second part of the formula
    denominator = sum(np.sqrt(1 / np.array(n_values)))
    
    # Calculate weights w_i
    weights = []
    for i in range(K):
        w_i = (1 / r_values[i]) * ((K * np.sqrt(1 / n_values[i])) / denominator)
        weights.append(w_i)
    
    # Calculate wMAE
    total_error = 0
    total_count = 0
    
    for i, (model, X_data, Y_data) in enumerate(zip(all_models, all_X_Data, all_Y_Data)):
        y_pred = model.predict(X_data)
        mae_i = np.abs(y_pred - Y_data).sum()
        total_error += weights[i] * mae_i
        total_count += len(Y_data)
    
    wmae = total_error / total_count
    return wmae, weights

Calculating and Printing Weighted `wMAE` for **Train** and **Test** Sets

In [ ]:
wmae_train, _ = compute_wmae(all_models, all_X_Train, all_Y_Train)
wmae_test, _ = compute_wmae(all_models, all_X_Test, all_Y_Test)

print(f"wMAE for Train data: {wmae_train}, wMAE for Test data: {wmae_test}")

# ✅Final Prediction

In [ ]:
feature_df = df_test.apply(create_features, axis = 1)
df_test = pd.concat([df_test, feature_df], axis = 1)
df_test = create_df(df_test)

test_id = df_test["id"]
df_test = df_test.drop("id", axis = 1)

submission = pd.DataFrame()
submission["id"] = test_id
for model, col in zip(all_models, col_names):
    submission[col] = model.predict(df_test)
submission    


In [ ]:
submission.to_csv("submission.csv", index = False, sep = ',')

# ✅Summary

In this first version of the pipeline, I implemented five separate regression models for the target properties:

* **XGBoost** for *Tg*
* **SVR** for *Tc*
* **LightGBM** for *FFV*, *Density*, and *Rg*

For training, I used three of the provided datasets (excluding the one containing only SMILES). Combining these sources might improve predictive accuracy on the test set, although the opposite effect is also possible due to potential data distribution differences.

Additionally, I incorporated molecular descriptors such as **MW**, **LogP**, **RotBonds**, **TPSA**, **FractionCSP3**, and **RingCount**, which were computed using **RDKit** based on the SMILES representations.

Thank you for reviewing my submission! I’d be happy to receive any comments and advice since I’m just beginning my journey in machine learning. I’ll keep working on optimizing the models and exploring additional features in future versions :)